In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, initializers
from tensorflow.keras.utils import plot_model
from IPython.display import Image
import tensorflow_datasets as tfds
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)
assert float(tf.__version__[:3]) >= 2.3

#tfds.list_builders()

In [ ]:
# cifar10 = tf.keras.datasets.cifar10
# (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
# train_images.shape

In [ ]:
train_data, info = tfds.load("cifar10", split="train[10%:]", with_info = True)
#print(info)
valid_data = tfds.load("cifar10", split="train[:10%]")
test_data = tfds.load("cifar10", split="test")

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteI60BU8/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteI60BU8/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.


In [ ]:
def parse_aug_fn(dataset):
    def zoom(x, sh = 299, sw = 299):
        x = tf.image.resize(x, (sh, sw))
        return x
    
    # 影像標準化
    x = tf.cast(dataset['image'], tf.float32) / 255.
    # 影像放大到224*224
    x = zoom(x)
    y = tf.one_hot(dataset['label'], 10)
    y = y
    
    return x, y

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 128
train_num = int(info.splits['train'].num_examples / 10) * 9
train_data = train_data.shuffle(train_num)
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data_1 = train_data.prefetch(buffer_size=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)
 
 
valid_data = valid_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)
 
test_data = test_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
test_data_1 = test_data.prefetch(buffer_size=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [ ]:
model_dir = '/content/drive/My Drive/Colab Notebooks/cifar10/models'
model = tf.keras.models.load_model(model_dir + "/Best-model-InceptionV3-L2.h5")

In [ ]:
#模型儲存目錄
model_dir = '/content/drive/My Drive/Colab Notebooks/cifar10/models'
if os.path.isdir(model_dir):
    print("已存在路徑「{}」".format(model_dir))
else:
    print("已建立路徑「{}」".format(model_dir))
    os.makedirs(model_dir)

已存在路徑「/content/drive/My Drive/Colab Notebooks/cifar10/models」


In [ ]:
#回調函數
log_dir = os.path.join('content','drive','My Drive','Colab Notebooks','cifar10','InceptionV3-L2')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + "/Best-model-InceptionV3-L2.h5", monitor='val_categorical_accuracy', mode='max')
# model_esp = keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=300, mode='max')

In [ ]:
# input_shape = (299, 299)
# base_model = tf.keras.applications.InceptionV3(include_top = False,
#                                               weights="imagenet",
#                                               pooling='avg',
#                                               input_shape=input_shape+(3,))
# model=tf.keras.Sequential([
#     base_model,
#     layers.Dense(1024,activation='relu', kernel_regularizer = 'l2'),
#     layers.Dense(10,activation='softmax')
# ])
 
# for layer in model.layers:
#     layer.trainable = False
# # 或者使用如下方法冻结所有层
# # model.trainable = False 
# model.layers[-1].trainable = True
# model.layers[-2].trainable = True
 


In [ ]:
# for x in model.trainable_weights:
#     print(x.name)

In [ ]:
# for x in model.non_trainable_weights:
#     print(x.name)

In [ ]:
#優化器、損失函數，指標函數
model.compile(keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 23,911,210
Trainable params: 2,108,426
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
# 有EarlyStopping
# history = model.fit(train_data, epochs=1000, validation_data = valid_data, callbacks=[model_cbk, model_mckp, model_esp])
history = model.fit(train_data, epochs=1000, validation_data = valid_data, callbacks=[model_cbk, model_mckp])

Epoch 1/1000
      1/Unknown - 0s 751us/step - loss: 0.4317 - categorical_accuracy: 0.8672WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


352/352 [==============================] - 522s 1s/step - loss: 0.4823 - categorical_accuracy: 0.8572 - val_loss: 0.5163 - val_categorical_accuracy: 0.8516
Epoch 2/1000
352/352 [==============================] - 514s 1s/step - loss: 0.4800 - categorical_accuracy: 0.8590 - val_loss: 0.5392 - val_categorical_accuracy: 0.8446
Epoch 3/1000
352/352 [==============================] - 514s 1s/step - loss: 0.4866 - categorical_accuracy: 0.8578 - val_loss: 0.5407 - val_categorical_accuracy: 0.8428
Epoch 4/1000
352/352 [==============================] - 514s 1s/step - loss: 0.4871 - categorical_accuracy: 0.8573 - val_loss: 0.5747 - val_categorical_accuracy: 0.8332
Epoch 5/1000
352/352 [==============================] - 515s 1s/step - loss: 0.4864 - categorical_accuracy: 0.8575 - val_loss: 0.5193 - val_categorical_accuracy: 0.8544
Epoch 6/1000
352/352 [==============================] - 514s 1s/step - loss: 0.4797 - categorical_accuracy: 0.8589 - val_loss: 0.5236 - val_categorical_accuracy: 0.8510

In [ ]:
model.evaluate(test_data)

In [ ]:
model.evaluate(train_data)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

In [ ]:
# Show model size in KBs.
float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

In [ ]:
# Show model size in KBs.
quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %dKBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_quantized_model)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
def evaluate_tflite_model(tflite_model):
  # Initialize TFLite interpreter using the model.
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_tensor_index = interpreter.get_input_details()[0]["index"]
  output = interpreter.tensor(interpreter.get_output_details()[0]["index"])
 
  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  i = 0
  accurate_count = 0
  labels = []
  for data in train_data_1.take(200):
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(np.array(data[0]), axis=0).astype(np.float32)
    interpreter.set_tensor(input_tensor_index, test_image)
 
    # Run inference.
    interpreter.invoke()
 
    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)
    
    labels.append(np.argmax(data[1]))
    # print("digit:",prediction_digits[i],"labels:",labels[i])
 
    
    i+=1
    if i%100 == 0:
      print('已讀取{}張圖片。'.format(i))
      # for index in range(len(prediction_digits)):
      #   if prediction_digits[index] == labels[index]:
      #     accurate_count += 1
      # accuracy = accurate_count * 1.0 / len(prediction_digits)
      # print(accuracy)
    
    
 
  # Compare prediction results with ground truth labels to calculate accuracy.
  
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == labels[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)
 
  return accuracy

In [ ]:
float_accuracy = evaluate_tflite_model(tflite_float_model)
print('Float model accuracy = %.4f' % float_accuracy)

In [ ]:
quantized_accuracy = evaluate_tflite_model(tflite_quantized_model)
print('Quantized model accuracy = %.4f' % quantized_accuracy)
print('Accuracy drop = %.4f' % (float_accuracy - quantized_accuracy))

In [ ]:
def evaluate_tflite_model(tflite_model):
  # Initialize TFLite interpreter using the model.
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_tensor_index = interpreter.get_input_details()[0]["index"]
  output = interpreter.tensor(interpreter.get_output_details()[0]["index"])
 
  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  i = 0
  accurate_count = 0
  labels = []
  for data in test_data_1.take(200):
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    # test_image[0] = np.expand_dims(np.array(test_image[0]), axis=0).astype(np.float32)
    test_image = np.expand_dims(np.array(data[0]), axis=0).astype(np.float32)
    interpreter.set_tensor(input_tensor_index, test_image)
 
    # Run inference.
    interpreter.invoke()
 
    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)
    labels.append(np.argmax(data[1]))
    # print("digit:",prediction_digits[i],"labels:",labels[i])
 
    
    i+=1
    if i%100 == 0:
      print('已讀取{}張圖片。'.format(i))
      # for index in range(len(prediction_digits)):
      #   if prediction_digits[index] == labels[index]:
      #     accurate_count += 1
      # accuracy = accurate_count * 1.0 / len(prediction_digits)
      # print(accuracy)
    
    
 
  # Compare prediction results with ground truth labels to calculate accuracy.
  
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == labels[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)
 
  return accuracy

In [ ]:
float_accuracy = evaluate_tflite_model(tflite_float_model)
print('Float model accuracy = %.4f' % float_accuracy)

In [ ]:
quantized_accuracy = evaluate_tflite_model(tflite_quantized_model)
print('Quantized model accuracy = %.4f' % quantized_accuracy)
print('Accuracy drop = %.4f' % (float_accuracy - quantized_accuracy))